<a href="https://colab.research.google.com/github/nelsongg/sentiment-analysis-imdb-reviews/blob/main/TensorFlow_Hub_Text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Hub - Text classification

- Based on: https://www.tensorflow.org/hub/tutorials/tf2_text_classification?hl=pt-br

# Importing the libraries

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
print('TensorFlow version: ', tf.__version__)
print('TensorFlow Hub version: ', hub.__version__)

TensorFlow version:  2.8.2
TensorFlow Hub version:  0.12.0


# Dataset

- Extracted from imdb: 0 - negative, 1 - positive

In [ ]:
train_data, test_data = tfds.load(name = 'imdb_reviews', split =['train', 'test'], batch_size=-1, as_supervised=True)
X_train, y_train = tfds.as_numpy(train_data)
X_test, y_test = tfds.as_numpy(test_data)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJJV24Z/imdb_reviews-train.tfrecord*...…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJJV24Z/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJJV24Z/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
X_train.shape, y_train.shape

((25000,), (25000,))

In [ ]:
X_test.shape, y_test.shape

((25000,), (25000,))

In [ ]:
X_train[0:2]

array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot 

In [ ]:
y_train[0:2]

array([0, 0])

In [ ]:
X_test[0:2]

array([b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.",
       b"A blackly comic tale of a down-trodden p

In [ ]:
y_test[0:2]

array([1, 1])

In [ ]:
np.unique(y_train, return_counts = True)

(array([0, 1]), array([12500, 12500]))

In [ ]:
np.unique(y_test, return_counts = True)

(array([0, 1]), array([12500, 12500]))

# Building and training the neural network

In [ ]:
# More models: https://tfhub.dev/s?module-type=text-embedding
model_path = 'https://tfhub.dev/google/nnlm-en-dim50/2'

In [ ]:
embedding_layer = hub.KerasLayer(model_path, input_shape = [], dtype = tf.string, trainable = True)

In [ ]:
# Word embedding: https://iaexpert.academy/2019/04/12/word-embedding-transformando-palavras-em-numeros/
embedding_layer(X_train[0:2])

<tf.Tensor: shape=(2, 50), dtype=float32, numpy=
array([[ 0.5423194 , -0.01190171,  0.06337537,  0.0686297 , -0.16776839,
        -0.10581177,  0.168653  , -0.04998823, -0.31148052,  0.07910344,
         0.15442258,  0.01488661,  0.03930155,  0.19772716, -0.12215477,
        -0.04120982, -0.27041087, -0.21922147,  0.26517656, -0.80739075,
         0.25833526, -0.31004202,  0.2868321 ,  0.19433866, -0.29036498,
         0.0386285 , -0.78444123, -0.04793238,  0.41102988, -0.36388886,
        -0.58034706,  0.30269453,  0.36308962, -0.15227163, -0.4439151 ,
         0.19462997,  0.19528405,  0.05666233,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201319 , -0.04418665, -0.08550781,
        -0.55847436, -0.2333639 , -0.20782956, -0.03543065, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862677,  0.7753425 , -0.07667087,
        -0.15752274,  0.01872334, -0.08169781, -0.3521876 ,  0.46373403,
        -0.08492758,  0.07166861, -0.00670818,  0.12686071, -0.19326551,
 

In [ ]:
model = tf.keras.Sequential()
model.add(embedding_layer)
# 50 -> 16 -> 1
model.add(tf.keras.layers.Dense(units = 16, activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer = 'adam', loss = tf.losses.BinaryCrossentropy(from_logits = True), metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 20, batch_size = 512, verbose = 1)

Epoch 1/20
49/49 [==============================] - 5s 38ms/step - loss: 0.6488 - accuracy: 0.5401
Epoch 2/20
49/49 [==============================] - 2s 37ms/step - loss: 0.4336 - accuracy: 0.7866
Epoch 3/20
49/49 [==============================] - 2s 37ms/step - loss: 0.2803 - accuracy: 0.8885
Epoch 4/20
49/49 [==============================] - 2s 37ms/step - loss: 0.1992 - accuracy: 0.9275
Epoch 5/20
49/49 [==============================] - 2s 37ms/step - loss: 0.1446 - accuracy: 0.9510
Epoch 6/20
49/49 [==============================] - 2s 37ms/step - loss: 0.1038 - accuracy: 0.9684
Epoch 7/20
49/49 [==============================] - 2s 37ms/step - loss: 0.0741 - accuracy: 0.9812
Epoch 8/20
49/49 [==============================] - 2s 38ms/step - loss: 0.0522 - accuracy: 0.9893
Epoch 9/20
49/49 [==============================] - 2s 38ms/step - loss: 0.0366 - accuracy: 0.9942
Epoch 10/20
49/49 [==============================] - 2s 37ms/step - loss: 0.0258 - accuracy: 0.9972
Epoch 11/

In [ ]:
results = model.evaluate(X_test, y_test)
print(results)

782/782 [==============================] - 4s 5ms/step - loss: 0.5610 - accuracy: 0.8494
[0.5609806776046753, 0.849399983882904]


# Predictions

In [ ]:
X_test[0:5]

array([b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.",
       b"A blackly comic tale of a down-trodden p

In [ ]:
y_test[0:5]

array([1, 1, 0, 0, 1])

In [ ]:
predictions = model.predict(X_test[0:5])

In [ ]:
predictions

array([[  3.0046253],
       [ -2.1390383],
       [ -4.4128532],
       [-10.96099  ],
       [  8.623336 ]], dtype=float32)

In [ ]:
predictions = tf.nn.sigmoid(predictions).numpy()
predictions

array([[9.5278263e-01],
       [1.0536000e-01],
       [1.1975398e-02],
       [1.7365808e-05],
       [9.9982017e-01]], dtype=float32)

In [ ]:
y_test[0:5]

array([1, 1, 0, 0, 1])

In [ ]:
predictions = (predictions >= 0.5)
predictions

array([[ True],
       [False],
       [False],
       [False],
       [ True]])